# EHSD Lab Exercise 1

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Microstrip Line

In [ ]:
C_matrix = np.array([
    [128.5, -12.0],
    [-12.0, 128.5]
]) # pF/m

L_matrix = np.array([
    [283.4, 52.2],
    [52.2, 283.4]
]) # nH/m

# Mode 1 (differential)
prop_speed_1 = 1.75463e8 # m/s
er_eff_1 = 2.919

# Mode 2 (common)
prop_speed_2 = 1.599689e8 # m/s
er_eff_2 = 3.512

In [ ]:
with open("../ltspice/circuit_model.asc", 'r', encoding='utf-8') as f:
    lines = f.readlines()

param_idx = []
param_lines = []
for i, line in enumerate(lines):
    if '.param' in line:
        param_idx.append(i)
        param_lines.append(line)

print("------ Current Parameters ------")
for line in param_lines:
    values = line.split('.param')[1].strip()
    print(values)

In [ ]:
# Case 1
component_values = {
    'C1': '7.5p',
    'R1': '10.9',
    # QFN
    'L1': '1.4154n',
    'R2': '96.9m',
    'C2': '0.1386p'
}

for i, line in enumerate(param_lines):
    mod_line = line.split('.param')[1].strip().split(' ')

    for key, value in component_values.items():
        for j, param in enumerate(mod_line):
            param = param.split('=')[0]
            if key == param:
                mod_line[j] = key + '=' + value
    mod_line = ' '.join(mod_line)
    
    lines[param_idx[i]] = lines[param_idx[i]].split('.param')[0] + '.param ' + mod_line + '\n'

param_lines = []
for i, line in enumerate(lines):
    if '.param' in line:
        param_lines.append(line)

print("------ New Parameters ------")
for line in param_lines:
    values = line.split('.param')[1].strip()
    print(values)

usr_input = input("Write? (y/n): ")
if usr_input == 'y':
    with open("../ltspice/circuit_model.asc", 'w', encoding='utf-8') as f:
        f.writelines(lines)